In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, concatenate, Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [9]:
x = train.drop(['id', 'digit', 'letter'], axis=1).values
x = x.reshape(-1, 28, 28, 1)
x = x/255

y_data = train['digit']
y = np.zeros((len(y_data), len(y_data.unique())))
for i, digit in enumerate(y_data):
    y[i, digit] = 1

In [10]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, shuffle=True, stratify=y)

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(1638, 28, 28, 1)
(410, 28, 28, 1)
(1638, 10)
(410, 10)


In [11]:
image_generator = ImageDataGenerator(width_shift_range=0.2, height_shift_range=0.2, 
                                     zoom_range=[0.75,1.25], brightness_range=[0.75,1.25], 
                                     shear_range=20)

In [12]:
def create_model(drop=0.2, optimizer='adam', padding='valid'):
    inputs = Input(shape=(28,28,1), name='input1')
    x1 = Conv2D(64, (3,3), activation='relu', padding=padding, name='conv1')(inputs)
    x1 = Dropout(drop)(x1)
    x1 = MaxPooling2D((2,2), name='pool1')(x1)
    x1 = Conv2D(64, (2,2), activation='relu', padding=padding, name='conv2')(x1)
    x1 = Dropout(drop)(x1)
    x1 = MaxPooling2D((2,2), name='pool2')(x1)
    x1 = Conv2D(128, (2,2), activation='relu', padding=padding, name='conv3')(x1)
    x1 = Dropout(drop)(x1)
    x1 = MaxPooling2D((2,2), name='pool3')(x1)
    x1 = Flatten(name='flat1')(x1)
    x2 = Dense(500, activation='relu', name='hidden1')(x1)
    x2 = Dropout(drop)(x2)
    x2 = Dense(100, activation='relu', name='hidden2')(x2)
    x2 = Dropout(drop)(x2)
    x2 = Dense(50, activation='relu', name='hidden3')(x2)
    outputs = Dense(10, activation='softmax', name='output')(x2)
    
    model = Model(inputs = inputs, outputs = outputs)
    model.compile(optimizer = optimizer, metrics = ['accuracy'], 
                  loss = 'categorical_crossentropy')
    
    return model

In [13]:
model = create_model(0.2, 'adam', 'same')

In [20]:
image_generator.fit(x_train, augment=True)

In [21]:
print(x_train.shape)

(1638, 28, 28, 1)


In [22]:
history = model.fit_generator(image_generator.flow(x_train, y_train, batch_size=64), 
                              steps_per_epoch=x_train.shape[0]/64, epochs=50, verbose=1, 
                              validation_data=image_generator.flow(x_val, y_val, batch_size=64), 
                              validation_steps=16)

Epoch 1/50
26/25 [==============================] - 5s 202ms/step - loss: 2.1171 - accuracy: 0.2277 - val_loss: 2.0883 - val_accuracy: 0.2489
Epoch 2/50
26/25 [==============================] - 5s 195ms/step - loss: 2.0884 - accuracy: 0.2411 - val_loss: 2.0721 - val_accuracy: 0.2711
Epoch 3/50
26/25 [==============================] - 5s 195ms/step - loss: 2.0743 - accuracy: 0.2454 - val_loss: 2.1206 - val_accuracy: 0.2437
Epoch 4/50
26/25 [==============================] - 5s 194ms/step - loss: 2.1267 - accuracy: 0.2100 - val_loss: 2.1012 - val_accuracy: 0.2584
Epoch 5/50
26/25 [==============================] - 5s 199ms/step - loss: 2.0614 - accuracy: 0.2381 - val_loss: 2.0448 - val_accuracy: 0.2753
Epoch 6/50
26/25 [==============================] - 5s 199ms/step - loss: 2.0674 - accuracy: 0.2442 - val_loss: 2.0472 - val_accuracy: 0.2848
Epoch 7/50
26/25 [==============================] - 6s 224ms/step - loss: 2.0244 - accuracy: 0.2546 - val_loss: 2.0599 - val_accuracy: 0.2901
Epoch 

In [ ]:
x_test = test.drop(['id', 'letter'], axis=1).values
x_test = x_test.reshape(-1, 28, 28, 1)
x_test = x_test/255
x_test.shape

In [ ]:
prediction = np.argmax(model.predict(x_test), axis=1)

In [ ]:
submission = pd.read_csv('data/submission.csv')
submission['digit'] = np.argmax(model.predict(x_test), axis=1)

In [ ]:
submission.to_csv('data/submission1.csv', index=False)